Transfer your Synthea CSV data to Azure blob storage using [Azure Storage Explorer](https://azure.microsoft.com/en-us/features/storage-explorer/).

Create a table to hold the account informationand access codes for your storage account. We put this in a table instead of hard-coding it in the notebook so the notebook is more shareable.

```
use default;

delete table if exists storage_account_name;

create table storage_account_secrets as
  select "my_storage_account_name" as storage_account_name,
      "<put your container name here>" as container_name,
      "<put your storage account key here>" as storage_account_key
;
```

In [0]:
storage_account_name == 'my_storage_account_name'

CSV_FOLDER = "/missouri/2021_07_19T16_51_37Z" # the folder on your storage accout where you have uploaded the Synthea CSV files

DB_NAME = 'missouri_test20y'  # data will be mounted in a folder of the same name

In [0]:
%python

# pull the first row of the secrets table into a dict
secrets = [r for i,r in spark.sql(f"select * from storage_account_secrets where storage_account_name == '{storage_account_name}'").toPandas().iterrows()][0]

DATA_SOURCE = "wasbs://{container_name}@{storage_account_name}.blob.core.windows.net".format(**secrets)

EXTRA_CONFIGS = {"fs.azure.account.key.{storage_account_name}.blob.core.windows.net".format(**secrets): secrets['storage_account_key']}

CURRENTLY_MOUNTED = {mount_info.mountPoint for mount_info in dbutils.fs.mounts()}

if f"/mnt/{DB_NAME}" in CURRENTLY_MOUNTED: 
  dbutils.fs.unmount(f"/mnt/{DB_NAME}")

dbutils.fs.mount(
 source = DATA_SOURCE  + CSV_FOLDER,
 mount_point = f"/mnt/{DB_NAME}",
 extra_configs = EXTRA_CONFIGS
)

[f.name for f in dbutils.fs.ls(f"/mnt/{DB_NAME}")]


In [0]:
import re

spark.sql(f"create database if not exists {DB_NAME}")
spark.sql(f"use {DB_NAME}") #"; select 1 as yes")

for file_info in dbutils.fs.ls(f"/mnt/{DB_NAME}"):
  table_name = re.sub('(.*)\\.csv$', '\\1', file_info.name).lower()
  print(f"creating table '{DB_NAME}.{table_name}' from file {file_info.path}")
  spark.read.options(header=True).csv(file_info.path).write.mode('overwrite').saveAsTable(table_name)